# Marks Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

In [ ]:
import seaborn as sns
#import statsmodels.api as sm

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_csv('data/marks.csv', sep=',')
df['W_Mark'] = df.Weight * df.Mark // 100
df['M_Mark'] = df.groupby(['Student','Module'])[ ['W_Mark'] ].transform(sum)
print(df.shape)
df

__1. What is the average mark per module?__

_For panda versions before panda 2.0_

- Group rows by 'Module'
- Compute the mean of the numeric columms, per 'Module' value
- Flatten the multi-index to 1 index using reset_index()
- Drop the unwanted 'Weight' column

Question is ambiguous, so all of Mark, Weighted Mark and Module Marks are shown.

In [ ]:
df.groupby(['Module']).mean(numeric_only=True).reset_index().drop(['Weight'], axis=1)

_For versions of panda from panda 2.0_

- Group rows by 'Module'
- Compute the mean of the numeric columms, per 'Module' value
- Flatten the multi-index to 1 index using reset_index()

In [ ]:
df.groupby(['Module']).agg({
  'Mark': 'mean',
  'W_Mark': 'mean',
  'M_Mark': 'mean'
}).reset_index()

Small refinement - rename the computed columns to clarify what aggreation function was used.

Apart from naming the computed aggregate columns, the data operations are the same.

In [ ]:
df.groupby(['Module']).agg(
  avgMark=("Mark", "mean"),
  avgW_Mark=("W_Mark", "mean"),
  avgM_Mark=("M_Mark", "mean")
).reset_index()

__2. What is the average mark per student?__

- Group rows by 'Student'
- Compute the mean of the numeric columms, per 'Student' value
- Flatten the multi-index to 1 index using reset_index()

Question is ambiguous, so Mark, Weighted Mark and Module Marks are shown.

Note that the code is almost identical to the code cell above, just swapping `'Module'` for `'Student'`.

In [ ]:
df.groupby(['Student']).agg(
  avgMark=("Mark", "mean"),
  avgW_Mark=("W_Mark", "mean"),
  avgM_Mark=("M_Mark", "mean")
).reset_index()

__3. What are the top two modules per student?__

The technique I showed in class (using `nlargest(2)`) worked in this case because duplicates did not affect the result.

However, (a variant of) the following code is needed for the next case, so the approach is used here too.

- Select the relevant columns in the dataframe (the order is significant)
- Drop duplicates (because `Weight` is not significant, but its removal means there are duplicates in the reduced dataframe)
- Sort By `Student` (increasing) and `M_Mark` (decreasing), so the result is almost there
- Group By `Student` (because we are looking for the Top 2 _per `Student`_)
- Apply `head(2)` (as the aggregation function) to return just the first 2 rows in each group of `Student`

In [ ]:
df[['Student', 'Module', 'M_Mark']].drop_duplicates().sort_values(['Student','M_Mark'],ascending = [True,False]).groupby('Student').head(2)

__4. What are the top two students per module?__

- Select the relevant columns in the dataframe (the order is significant)
- Drop duplicates (because `Weight` is not significant, but its removal means there are duplicates in the reduced dataframe)
- Sort By `Module` (increasing) and `M_Mark` (decreasing), so the result is almost there
- Group By `Module` (because we are looking for the Top 2 _per `Module`_)
- Apply `head(2)` (as the aggregation function) to return just the first 2 rows in each group of `Module`

In [ ]:
df[['Module', 'Student', 'M_Mark']].drop_duplicates().sort_values(['Module','M_Mark'],ascending = [True,False],).groupby('Module').head(2)

__5. On average, do students achieve higher marks in exams than in practical (i.e., non-exam) assessments?__

- First we add an AssessType column that distinguishes between Exam and Practical assessments

In [ ]:
df['AssessType'] = np.where(df['Deliverable'] == 'Exam', 'Exam', 'Practical')
df

Note that we have a suitable grouping column (`AssessType`), we can apply the aggregate function to the only remaining column (`Mark`) in the projected dataframe.

In [ ]:
df[['AssessType','Mark']].groupby('AssessType').mean(numeric_only=True).reset_index()

Note that this is just the simplest interpretation. A more complex interpretation would use weighting to estimate whether Exams or Practicals are contributing relatively more to overall marks.